In [78]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)


In [135]:
def mars_news():

    # Gather the latest Mars news from NASA
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)

    # Scrape the page
    soup = BeautifulSoup(browser.html, 'lxml')

    hl_list = [] # Store the headlines
    sl_list = [] # Store the straplines

    # News articles are in a div tag class list_text
    article = soup.find_all('div', class_='list_text')

    # Loop through returned results
    for item in article:

        # Error handling
        try:
            # Grab the headline
            headline = item.find('a').get_text()
            # Grab the strapline
            strapline = item.find('div', class_='article_teaser_body').get_text()
            # Append to the lists
            hl_list.append(headline)
            sl_list.append(strapline)

            # Print results only if title, price, and link are available
            if (headline and strapline):
                print('-------------')
                print(headline)
                print(strapline)
        except AttributeError as e:
            print(e)

In [136]:
mars_news()


-------------
Six Things to Know About NASA's Opportunity Rover
Opportunity's mission is complete. Here are highlights from its time on Mars.
-------------
NASA's Opportunity Rover Mission on Mars Comes to End
NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet.
-------------
NASA's InSight Prepares to Take Mars' Temperature
The lander's second instrument is on the ground. But its ‘mole’ — a burrowing probe — has a long way to go.
-------------
NASA to Share Results of Effort to Recover Mars Rover
NASA will discuss the status of its Mars Exploration Rover Opportunity in a media briefing at 11 a.m. PST (2 p.m. EST) Wednesday, Feb. 13, from the Jet Propulsion Laboratory.
-------------
NASA's MAVEN Spacecraft Shrinking its Mars Orbit to Prepare for Mars 2020 Rover
The MAVEN spacecraft today is starting a campaign to tighten its orbit around Mars to prepare to serve as 

In [145]:
def mars_image():
# Visit the JPL site 
    image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    base_url = image_url.split('/spaceimages')[0]
    browser.visit(image_url)

    # Click to expand fancybox and get the full image
    try:
        browser.click_link_by_partial_text('FULL IMAGE')
    except ElementDoesNotExist:
        print("Error with featured image")

    html = browser.html
    # Scrape out the image link
    soup2 = BeautifulSoup(html, 'lxml')
    image_wrapper = soup2.find('div', class_='fancybox-inner')
    image_tag = image_wrapper.find('img')
    time.sleep(3)
    # After it loads
    ref_link = image_tag['src']
    featured_image_url = base_url + ref_link
    return featured_image_url

In [146]:
mars_image()

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16883_ip.jpg'

In [151]:
def mars_weather():
# Visit the Mars Weather Twitter page
    twitter_url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(twitter_url)
    html = browser.html

    # Scrape out some tweets
    soup3 = BeautifulSoup(html, 'lxml')
    tweets = soup3.find_all('div', class_='js-tweet-text-container', limit=10)
    # To get around potential retweets,
    # go through the top 10 tweets and find one that looks like weather
    for item in tweets:
        try:
            if item.p.text.split(' ')[0] == 'Sol':
                mars_weather = item.p.text.split('pic.twitter')[0]
                return mars_weather
                # Exit the loop if one is found that looks like weather
                break
        except AttributeError:
            print("Error with tweet")
    

In [152]:
mars_weather()

'Sol 2319 (2019-02-13), high -17C/1F, low -72C/-97F, pressure at 8.12 hPa, daylight 06:46-18:52'

In [153]:
def mars_facts():
# Visit the Mars Space Facts page
    facts_url = 'http://space-facts.com/mars/'
    tables = pd.read_html(facts_url)
    df = tables[0]
    df.columns=['fact','data']
    df.set_index('fact')
    facts_table = df.to_html()
    return facts_table

In [155]:
mars_facts()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>fact</th>\n      <th>data</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>

In [156]:
def mars_hemispheres():
# Get the results from Mars Hemispheres image search and store in results
    h_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    base_url = h_url.split('/search')[0]
    # Go to the page
    browser.visit(h_url)
    soup4 = BeautifulSoup(browser.html, 'html.parser')
    results = soup4.find_all('div', class_='description')
    # A blank list for the image name and URLS
    hemisphere_image_urls = []

    # Iterate through the soup results and store the information as a list of dictionaries
    for item in results:
        link_dict = {} # A temporary dictionary
        link = base_url + item.find('a')['href']
        # Visit each link in the results to get full image information
        try:
            browser.visit(link)
    #         print(f'Going to the new page...{link}') #debugging
            time.sleep(1)
    #         print('success') #debugging
            #Now scrape the page looking for the jpg image with text 'Sample'
            soup = BeautifulSoup(browser.html, 'html.parser')
            img_list = soup.find('a', text='Sample')
            # Storing the data in the dictionary 
            link_dict['title'] = soup.find('h2', class_='title').text
            link_dict['img_url'] = img_list['href']
            # then appending that to the main list
            hemisphere_image_urls.append(link_dict)
#             print('updated') #Debugging
        except ElementDoesNotExist:
            print("Error with featured image")

    return hemisphere_image_urls
    
    

In [157]:
mars_hemispheres()

updated
updated
updated
updated


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

updated
updated
updated
updated
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
